In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel


# Load the base Llama 2 7B model
base_model_id = "NousResearch/Llama-2-7b-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id, 
    quantization_config=bnb_config, 
    device_map="auto",
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True)

# Load the QLoRA adapter
path_llm_model = "/nfs/home/wzr5897/amazon_lora_llm/llama2-7b-AmazonVPC-finetune/checkpoint-500"
ft_model = PeftModel.from_pretrained(base_model, path_llm_model)

def clean_output(output):
    # Example of cleaning up output - adjust as needed based on the actual output format
    answer_start = output.find("### Answer:")
    if answer_start != -1:
        return output[answer_start + len("### Answer:"):].strip()
    return output

# Function to generate LLM answer
def generate_llm_answer(question):
    #question = question + " #"
    model_input = tokenizer(question, return_tensors="pt").to("cuda")
    ft_model.eval()
    with torch.no_grad():
        raw_output = tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=300)[0], skip_special_tokens=True)
        return clean_output(raw_output)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/nfs/home/wzr5897/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/nfs/home/wzr5897/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [3]:
# Read CSV file
df = pd.read_csv("test.csv")

# Generate LLM answers for each question
df['llm_answer'] = df['question'].apply(generate_llm_answer)

# Save the new DataFrame to a CSV file
df.to_csv("output_test.csv", index=False)


In [5]:
pd.set_option('display.max_colwidth', None)  # This will remove the truncation of column content
df

,question,answer,llm_answer
0,What are AWS managed policies for Network Access Analyzer and how do they function?,"AWS managed policies for Network Access Analyzer are predefined permission sets that make it easier to assign permissions to users, groups, and roles within Amazon Virtual Private Cloud. AWS services are responsible for maintaining and periodically updating these managed policies to support new features or operations, ensuring that permissions do not get removed and existing permissions do not break. Notably, these managed policies cannot be modified by users. For instance, the AmazonVPCNetworkAccessAnalyzerFullAccessPolicy is an AWS managed policy that provides comprehensive permissions for creating, analyzing, and deleting Network Access Scopes, as well as describing various network path resources like firewalls and gateways. Additionally, AWS also offers managed policies designed for specific job functions across multiple services.","AWS managed policies for Network Access Analyzer are pre-built sets of permissions that simplify the process of managing access to Network Access Analyzer resources. These policies are created and maintained by AWS and are designed to cover common use cases, such as allowing users to create, view, and delete network access analyses. Users can attach these policies to their IAM identities to grant them the necessary permissions without having to create their own policies. The policies are stored in the AWS managed policies section of the IAM console and can be edited or deleted as needed. AWS recommends using managed policies for simplified access management and to ensure compliance with various AWS services."
1,"What are the default quotas and their adjustability for AWS Network Access Analyzer, and how does one view or request changes to these quotas?","Your AWS account comes with default quotas for the Network Access Analyzer, which have replaced the term ""limits."" There are several quotas associated with the Network Access Analyzer: Access scopes have a default quota of 1,000, and they are adjustable. Access scope analyses also have an adjustable default quota set at 10,000, and the number of concurrent access scope analyses is capped at 25, which is also adjustable. The findings per scope analysis have a fixed limit of 1,000 and are not adjustable. To view these quotas, you should navigate to the Service Quotas console, select ""AWS services,"" and then choose ""Network Insights."" If you need to request an increase for a quota, you can follow the guidelines provided in the ""Requesting a quota increase"" section of the Service Quotas User Guide. It's important to note that every analysis includes all network interfaces in the account and region, and the analysis will time out after one hour and thirty minutes.","By default, AWS Network Access Analyzer comes with a set of quotas that can be adjusted to accommodate specific needs. The quotas include a maximum of 100,000 network access analyses per Region per day, a maximum of 100,000 network access analyses per account per Region per day, and a maximum of 100,000 network access analyses per account per Region per month. These quotas can be increased by submitting a request to the AWS Quotas console. Additionally, the Network Access Analyzer console provides a dashboard that displays the current usage of these quotas, including the number of analyses completed and the number of analyses remaining within the quota limits. Users can also view the quota history, which includes the number of analyses completed and the number of analyses remaining within the quota limits for the past 90 days. If a user needs to increase the quotas, they can follow the instructions provided in the AWS Quotas User Guide to request an increase. It's important to note that quota increases are subject to approval and may take several days to process. The Network Access Analyzer console"
2,When was the initial release for the Network Access Analyzer accor